In [13]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from emnist import extract_training_samples, extract_test_samples

# Load the EMNIST dataset
x_train, y_train = extract_training_samples('letters')
x_test, y_test = extract_test_samples('letters')

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = y_train - 1
y_test = y_test - 1

# Ensure that all labels are in the valid range [0, 25]
assert np.min(y_train) >= 0 and np.max(y_train) <= 25
assert np.min(y_test) >= 0 and np.max(y_test) <= 25

# Reshape the data to have a single channel (grayscale)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Create a CNN model
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(26, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Fitting the model
model.fit(x_train, y_train, epochs=10)
model.save('alphabets_cnn.model')

Epoch 1/10
3120/3120 [==============================] - 47s 15ms/step - loss: 0.4650 - accuracy: 0.8547
Epoch 2/10
3120/3120 [==============================] - 48s 15ms/step - loss: 0.2394 - accuracy: 0.9214
Epoch 3/10
3120/3120 [==============================] - 49s 16ms/step - loss: 0.1914 - accuracy: 0.9356
Epoch 4/10
3120/3120 [==============================] - 46s 15ms/step - loss: 0.1587 - accuracy: 0.9448
Epoch 5/10
3120/3120 [==============================] - 47s 15ms/step - loss: 0.1354 - accuracy: 0.9509
Epoch 6/10
3120/3120 [==============================] - 45s 14ms/step - loss: 0.1172 - accuracy: 0.9560
Epoch 7/10
3120/3120 [==============================] - 45s 15ms/step - loss: 0.1036 - accuracy: 0.9604
Epoch 8/10
3120/3120 [==============================] - 49s 16ms/step - loss: 0.0918 - accuracy: 0.9636
Epoch 9/10
3120/3120 [==============================] - 51s 16ms/step - loss: 0.0830 - accuracy: 0.9669
Epoch 10/10
3120/3120 [==============================] - 51s 16m

INFO:tensorflow:Assets written to: alphabets_cnn.model\assets


In [17]:
# Evaluating the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

780/780 [==============================] - 3s 3ms/step - loss: 0.2757 - accuracy: 0.9300
Test Loss: 0.2757, Test Accuracy: 0.9300


1/1 [==============================] - 0s 47ms/step
				  The letter is probably:
						' Q '


In [19]:
# Load and use the model
model = tf.keras.models.load_model('alphabets_cnn.model')

# Load and preprocess the image
img = cv2.imread("alphabets/test.png", cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28, 28))
img = img / 255.0
img = img.reshape(1, 28, 28, 1)

# Make prediction
prediction = model.predict(img)
predicted_label = np.argmax(prediction)
recognized_character = chr(predicted_label + ord('A'))

print("\t\t\t\t  The letter is probably:\n\t\t\t\t\t\t'", recognized_character, "'")
plt.imshow(img[0, :, :, 0], cmap=plt.cm.binary)
plt.show()